https://github.com/earthpulse/eotdl/issues/190


1. find and explore the EuroCropsDataset, stage it in the EODTL workspace
2. filter the EuroCropsDataset dataset using EOTDL functionality, to create a subset of parcels,
   e.g., 8 crop classes, each with 1000 examples, for one country
3. run feature engineering with openEO, creating temporal metrics from a S1 and S2 time series (temporally optimised for crops classe of interest). Store feature engineering process graph with the training datsets in EOTDL
4. Use EOTDL functionality to train a model (for this the features need to be retrieved..). Store the model along with the openEO process graph in EOTDL.
5. Use the model to run inference (from within EOTDL?) in an openEO platform such as CDSE or openEO platform. Make use of the feature engineering process graph stored along with the EOTDL model.

## 1 Ingest EuroCrops to EOTDL as Q1/Q2

We already have Q0 https://www.eotdl.com/datasets/EuroCrops/

Need to create Q1/Q2 for step 2.

## 2. Filter EuroCropsDataset

Use STAC/GeoDB to filter the dataset.
This will return a STAC catalog with the filtered items.

## 3. Feature Engineering with openEO

We need to see how openEO can use the items in the resulting catalog.

Generate jsons with the openEO process graphs and save in EOTDL.

Get the resulting features.

Save features as dataset in EOTDL for training.

## 4. Train a model with EOTDL

Use dataset with features for training (use exsiting examples).

Ingest the resulting model in EOTDL (onnx)

## 5. Run inference with EOTDL

Use model in EOTDL to run inference with openEO in CDSE or other platform.